DMD FUNCTIONS - FAT matrix X - for ONLINE DMD comparison - Zhang, Rowley,...

In [68]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from scipy import linalg

In [69]:
def GEDMD(X, Y, tol=1e-12, type: str=None, k=-1, weights=None): 
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=X.shape[1]; N=X.shape[0]
    D = np.linalg.norm(X, axis=0)
    X = X/D; Y=Y/D
    if(weights is not None):
        X = X*weights; Y=Y*weights
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) 
    V=np.conjugate(V.T)
    if(k==-1):
        k=m
        for i in range(1,min(N,m)):
            if(Sigma[i]<=Sigma[0]*tol):
                k=i
                break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    B_k = Y @ (V/Sigma)
    S_k = np.conjugate(U.T)@B_k
    Lambda, W = np.linalg.eig(S_k)
    Z = U@W
    if(type=="exact"):
        Z = B_k@W
    #r = np.linalg.norm(B_k@W - Z*Lambda, axis=0)
    r = np.linalg.norm(B_k@W - Lambda*Z, axis=0)

    return Z, Lambda, r
    return Z, Lambda

In [70]:
def fGEDMDQ_multiple_trajectories(X, Y, tol=1e-12, type: str=None):
    m = X.shape[0]
    M = np.bmat([X.T, Y.T])
    Q, L = np.linalg.qr(M)
    L = L.T
    Z, Lambda, r = GEDMD(L[:m,:m], L[m:,:m], tol, type)
    #Z, Lambda= GEDMD(L[:m,:m], L[m:,:m], tol, type)
    return Z, Lambda, r, Q, L 
    return Z, Lambda, Q, L

In [71]:
def fDMD_added_snap(v, Q, L, tol=1e-12, type: str=None, k=-1, weights=None, ngram=5):
    #v tipa 1 x 2m => prvih m je zadnji element u Y, drugih m je novi element
    # ngram - treba li tu reortogonalizacija?
    m=Q.shape[1]//2; n=Q.shape[0]
    print(m)
    R = np.bmat([[L.T],[v.T]])
    U, T = np.linalg.qr(R)
    T=T.T #novi L - sad je T donjetrokutasto
    Q = np.bmat([[Q@U[:2*m,:]],[U[-1,:]]])
    #novi Z i Lambda i r
    Z, Lambda, r = GEDMD(T[:m,:m], T[m:,:m], tol, type)
    return Z, Lambda, r, Q, T
    return Z, Lambda, Q, T



In [72]:
### Ovo bi trebalo biti isto?

def DMD_alpha_for_reconstruction(X, Z, indices, L, weights=None):
    """X = snapshotovi - prvih m (bez m+1-vog) - dakle X, a ne S
    Z = modes,
    indices = indices which we want to work with - from 1/r graph
    weights = np.array tip ili lista"""
    #treba li se formirati Vandermondeova matrica?
    m=X.shape[1]; l=indices.shape[0]
    Z = Z[:,indices]
    Q, R = np.linalg.qr(Z) # Q je tipa duljina_snapshota(N) x l, R je tipa lxl
    if(weights is None):
        weights=np.ones((m)).reshape(-1)
    weights=np.array(weights)
    pom=np.vander(L[indices], m, increasing=True)*weights
    alpha= np.multiply(np.conj(R.T)@R, np.conj(pom @ np.conj(pom.T)))
    G = (np.conj(Q.T) @ X)[:l, :]  
    alpha = scipy.linalg.solve(alpha, np.multiply(np.conj(pom),(np.conj(R.T)@G))@np.ones((m, 1)), assume_a='pos')
    #alpha = scipy.linalg.solve(alpha, np.multiply(np.conj(pom),(np.conj(R.T)@G*weights))@np.ones((m, 1)), assume_a='pos')
    return Z, L[indices], alpha.reshape(-1)

def DMD_reconstruction(X, Z, indices, L, times, weights=None, real=True): #mozda da prima vektor napraviti..
    """X = snapshotovi - prvih m (bez m+1-vog) - dakle X, a ne S,
    Z = dmd modes (returned from some version of DMD, ex. GEDMDQ)
    L = dmd eigs (returned from some version of DMD, ex. GEDMDQ)
    time = integer, which datasnapshot you want to reconstruct/predict"""
    Z_l, L, alpha = DMD_alpha_for_reconstruction(X, Z, indices, L, weights)
    num=np.asarray(times).shape[0]
    recs = np.empty((Z_l.shape[0], num), dtype='complex_')
    for i in range(num):
        recs[:,i] = Z_l@(L**(times[i])*alpha) #mislim da je times[i] jer ovdje krecemo od 0, za razliku od matlaba gdje krecemo od 1
    if real:
        return np.real(recs) #ako sve realno, ovo ce biti realno za svaki i, samo ce zapis biti u obliku kompleksnog; zato saljemo np.real(recs)!
    else:
        return recs


JEDNOSTAVNA PROVJERA

In [73]:
A=np.random.rand(4,4); A=A/np.linalg.norm(A)
x=np.ones((4,1))
y=A@x
Xevi = np.bmat([[np.copy(x)],[np.copy(y)]])
x=y
for i in range(30):
    y=A@x
    Xevi = np.bmat([Xevi, np.bmat([[np.copy(x)],[np.copy(y)]])])
    x=y

np.set_printoptions(2)
Xevi

matrix([[1.  , 0.52, 0.39, 0.32, 0.27, 0.24, 0.21, 0.18, 0.16, 0.14,
         0.12, 0.11, 0.09, 0.08, 0.07, 0.06, 0.05, 0.05, 0.04, 0.04,
         0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01,
         0.01],
        [1.  , 0.97, 0.88, 0.77, 0.68, 0.59, 0.52, 0.45, 0.39, 0.34,
         0.3 , 0.26, 0.23, 0.2 , 0.17, 0.15, 0.13, 0.12, 0.1 , 0.09,
         0.08, 0.07, 0.06, 0.05, 0.05, 0.04, 0.03, 0.03, 0.03, 0.02,
         0.02],
        [1.  , 1.14, 0.99, 0.87, 0.76, 0.66, 0.58, 0.5 , 0.44, 0.38,
         0.34, 0.29, 0.26, 0.22, 0.2 , 0.17, 0.15, 0.13, 0.11, 0.1 ,
         0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.04, 0.03, 0.03, 0.03,
         0.02],
        [1.  , 0.87, 0.81, 0.72, 0.63, 0.55, 0.48, 0.42, 0.37, 0.32,
         0.28, 0.24, 0.21, 0.19, 0.16, 0.14, 0.12, 0.11, 0.09, 0.08,
         0.07, 0.06, 0.05, 0.05, 0.04, 0.04, 0.03, 0.03, 0.02, 0.02,
         0.02],
        [0.52, 0.39, 0.32, 0.27, 0.24, 0.21, 0.18, 0.16, 0.14, 0.12,
         0.11, 0.09, 0.08, 0.07, 0.06, 

In [74]:
np.linalg.eig(A)

(array([ 0.87,  0.25, -0.05, -0.14]),
 array([[ 0.22,  0.83, -0.32, -0.18],
        [ 0.55, -0.33,  0.19, -0.29],
        [ 0.62, -0.12,  0.74,  0.81],
        [ 0.51, -0.44, -0.56, -0.48]]))

In [75]:
GEDMD(Xevi[:4,:20], Xevi[4:,:20]) #iste sv. vrijednosti

(matrix([[-0.22, -0.83,  0.32, -0.18],
         [-0.55,  0.33, -0.19, -0.29],
         [-0.62,  0.12, -0.74,  0.81],
         [-0.51,  0.44,  0.56, -0.48]]),
 array([ 0.87,  0.25, -0.05, -0.14]),
 array([0.45, 1.44, 0.39, 0.44]))

In [76]:
Z, Lambda, r, Q, L = fGEDMDQ_multiple_trajectories(Xevi[:4,:20], Xevi[4:,:20])
Lambda #iste sv. vrij.
r

array([0.47, 1.38, 0.66, 0.61])

In [78]:
Z, Lambda, r, Q, L = fDMD_added_snap(Xevi[:,20], Q, L)

4


In [81]:
r

array([0.47, 1.38, 0.66, 0.61])

In [79]:
m=4
indices = np.array([i for i in range(4)]); times=[i for i in range(31)]
reconstructed=DMD_reconstruction(Xevi[:m,:], Z, indices, Lambda, times)

In [80]:
reconstructed-Xevi[:m,:]

matrix([[ 8.88e-16,  1.11e-16, -1.11e-16, -1.11e-16, -5.55e-17,
         -2.78e-17, -2.78e-17, -5.55e-17, -5.55e-17, -2.78e-17,
         -1.39e-17, -1.39e-17, -1.39e-17,  0.00e+00,  0.00e+00,
         -6.94e-18,  0.00e+00,  0.00e+00,  0.00e+00,  0.00e+00,
          3.47e-18,  6.94e-18,  6.94e-18,  3.47e-18,  6.94e-18,
          6.94e-18,  3.47e-18,  3.47e-18,  1.73e-18,  3.47e-18,
          3.47e-18],
        [-1.11e-16,  3.33e-16,  1.11e-16,  2.22e-16,  1.11e-16,
          0.00e+00,  1.11e-16,  1.11e-16,  1.11e-16,  1.11e-16,
          1.67e-16,  5.55e-17,  1.39e-16,  5.55e-17,  8.33e-17,
          5.55e-17,  1.11e-16,  8.33e-17,  5.55e-17,  6.94e-17,
          5.55e-17,  6.94e-17,  4.86e-17,  4.16e-17,  4.86e-17,
          2.78e-17,  2.78e-17,  2.43e-17,  2.43e-17,  2.08e-17,
          2.08e-17],
        [ 0.00e+00,  0.00e+00,  4.44e-16,  3.33e-16,  3.33e-16,
          2.22e-16,  3.33e-16,  2.22e-16,  2.22e-16,  2.78e-16,
          1.67e-16,  1.67e-16,  2.22e-16,  1.39e-16,  1.67e-16